In [ ]:
import numpy as np
from nltk.tokenize import RegexpTokenizer
from keras.models import Sequential, load_model
from keras.layers import LSTM
from keras.layers.core import Dense, Activation
from keras.optimizers import RMSprop
import matplotlib.pyplot as plt
import pickle
import heapq

In [ ]:
text = open('1661-0.txt', encoding='utf8').read().lower()
print('corpus length:', len(text))

corpus length: 581533


In [ ]:
tokenizer = RegexpTokenizer(r'\w+')
words = tokenizer.tokenize(text)

In [ ]:
unique_words = np.unique(words)
unique_word_index = dict((c, i) for i, c in enumerate(unique_words))

In [ ]:
WORD_LENGTH = 5
prev_words = []
next_words = []
for i in range(len(words) - WORD_LENGTH):
    prev_words.append(words[i:i + WORD_LENGTH])
    next_words.append(words[i + WORD_LENGTH])

In [ ]:
X = np.zeros((len(prev_words), WORD_LENGTH, len(unique_words)), dtype=bool)
Y = np.zeros((len(next_words), len(unique_words)), dtype=bool)

In [ ]:
for i, each_words in enumerate(prev_words):
    for j, each_word in enumerate(each_words):
        X[i, j, unique_word_index[each_word]] = 1
    Y[i, unique_word_index[next_words[i]]] = 1

In [ ]:
model = Sequential()
model.add(LSTM(128, input_shape=(WORD_LENGTH, len(unique_words))))
model.add(Dense(len(unique_words)))
#model.add(Dense(len(unique_words)/2))
model.add(Activation('softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 128)               4256768   
                                                                 
 dense (Dense)               (None, 8185)              1055865   
                                                                 
 activation (Activation)     (None, 8185)              0         
                                                                 
Total params: 5,312,633
Trainable params: 5,312,633
Non-trainable params: 0
_________________________________________________________________


In [ ]:
from tensorflow import keras
from keras.utils.vis_utils import plot_model

keras.utils.plot_model(model, to_file='plot.png', show_layer_names=True)  # type: ignore

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


In [ ]:
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(learning_rate=0.02), metrics=['accuracy'])
history = model.fit(X, Y, validation_split=0.2, batch_size=128, epochs=100, shuffle=True).history

Epoch 1/100
845/845 [==============================] - 190s 221ms/step - loss: 6.1074 - accuracy: 0.1151 - val_loss: 8.2134 - val_accuracy: 0.1218
Epoch 2/100
845/845 [==============================] - 163s 193ms/step - loss: 5.9055 - accuracy: 0.1558 - val_loss: 8.0750 - val_accuracy: 0.1273
Epoch 3/100
845/845 [==============================] - 155s 184ms/step - loss: 5.4202 - accuracy: 0.1931 - val_loss: 8.4537 - val_accuracy: 0.1181
Epoch 4/100
845/845 [==============================] - 174s 206ms/step - loss: 4.8675 - accuracy: 0.2402 - val_loss: 9.5028 - val_accuracy: 0.1319
Epoch 5/100
845/845 [==============================] - 185s 219ms/step - loss: 4.2908 - accuracy: 0.2927 - val_loss: 10.3577 - val_accuracy: 0.1227
Epoch 6/100
845/845 [==============================] - 173s 204ms/step - loss: 3.7253 - accuracy: 0.3524 - val_loss: 10.7306 - val_accuracy: 0.1245
Epoch 7/100
845/845 [==============================] - 190s 224ms/step - loss: 3.2215 - accuracy: 0.4109 - val_loss:

In [ ]:
model.save('nwp.h5')
pickle.dump(history, open("history.p", "wb"))

model = load_model('nwp.h5')
history = pickle.load(open("history.p", "rb"))

In [ ]:
def prepare_input(text):
    x = np.zeros((1, WORD_LENGTH, len(unique_words)))
    for t, word in enumerate(text.split()):
        print(word)
        x[0, t, unique_word_index[word]] = 1
    return x

In [ ]:
def sample(preds, top_n=3):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds)
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    return heapq.nlargest(top_n, range(len(preds)), preds.take)

In [ ]:
def predict_completions(text, n=3):
    if text == "":
        return("0")
    x = prepare_input(text)
    preds = model.predict(x, verbose=0)[0]  # type: ignore
    next_indices = sample(preds, n)
    return [unique_words[idx] for idx in next_indices]

In [ ]:
q = "Your life will never be there in the same situation again"
print("correct sentence: ",q)
seq = " ".join(tokenizer.tokenize(q.lower())[0:5])
print("Sequence: ",seq)
print("next possible words: ", predict_completions(seq, 3))

correct sentence:  Your life will never be there in the same situation again
Sequence:  your life will never be
your
life
will
never
be
next possible words:  ['upon', 'carried', 'shoulders']
